# TSA Chapter 7: Case Study - VECM Forecast

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_case_forecast/TSA_ch7_case_forecast.ipynb)

Out-of-sample VECM forecasts with confidence intervals for FRED GS10 and TB3MS interest rates, 24-step horizon.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas_datareader yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import VECM
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Fetch real data from FRED
start, end = '2000-01-01', '2024-12-31'
gs10 = web.DataReader('GS10', 'fred', start, end).dropna()
tb3ms = web.DataReader('TB3MS', 'fred', start, end).dropna()
data = pd.DataFrame({'GS10': gs10['GS10'], 'TB3MS': tb3ms['TB3MS']}).dropna()
print(f'Period: {data.index[0].strftime("%Y-%m")} to {data.index[-1].strftime("%Y-%m")}, n={len(data)}')

# Fit VECM on full data and forecast 24 steps
n_forecast = 24
vecm_model = VECM(data, k_ar_diff=2, coint_rank=1)
vecm_res = vecm_model.fit()

# Point forecast and confidence intervals
fcast = vecm_res.predict(steps=n_forecast)
fcast_lower, fcast_upper = vecm_res.predict_ci(steps=n_forecast, alpha=0.05)

# Build forecast date index
last_date = data.index[-1]
fcast_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=n_forecast, freq='MS')

# Compute forecast spread
spread_hist = data['GS10'] - data['TB3MS']
spread_fcast = fcast[:, 0] - fcast[:, 1]
spread_lower = fcast_lower[:, 0] - fcast_upper[:, 1]  # conservative CI
spread_upper = fcast_upper[:, 0] - fcast_lower[:, 1]

# Show last 60 months of history
n_hist = 60

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Panel 1: Rate forecasts
hist_idx = data.index[-n_hist:]
axes[0].plot(hist_idx, data['GS10'].iloc[-n_hist:], color=BLUE, lw=1, label='GS10 (actual)')
axes[0].plot(hist_idx, data['TB3MS'].iloc[-n_hist:], color=RED, lw=1, label='TB3MS (actual)')
axes[0].plot(fcast_dates, fcast[:, 0], color=BLUE, lw=1.5, ls='--', label='GS10 (forecast)')
axes[0].plot(fcast_dates, fcast[:, 1], color=RED, lw=1.5, ls='--', label='TB3MS (forecast)')
axes[0].fill_between(fcast_dates, fcast_lower[:, 0], fcast_upper[:, 0], color=BLUE, alpha=0.1)
axes[0].fill_between(fcast_dates, fcast_lower[:, 1], fcast_upper[:, 1], color=RED, alpha=0.1)
axes[0].axvline(last_date, color=GRAY, ls=':', lw=0.8)
axes[0].set_title('VECM Rate Forecasts (24 months)', fontweight='bold')
axes[0].set_ylabel('Rate (%)')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False, fontsize=8)

# Panel 2: Spread forecast
axes[1].plot(hist_idx, spread_hist.iloc[-n_hist:], color=GREEN, lw=1, label='Spread (actual)')
axes[1].plot(fcast_dates, spread_fcast, color=GREEN, lw=1.5, ls='--', label='Spread (forecast)')
axes[1].fill_between(fcast_dates, spread_lower, spread_upper, color=GREEN, alpha=0.1)
axes[1].axhline(0, color='black', ls='-', lw=0.5)
axes[1].axvline(last_date, color=GRAY, ls=':', lw=0.8)
axes[1].set_title('Term Spread Forecast', fontweight='bold')
axes[1].set_ylabel('Spread (%)')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

plt.tight_layout()
save_chart(fig, 'ch7_case_forecast')
plt.show()